In [ ]:
# Projeto: Previsão do Valor Total de Pedidos (Abril/2025)
# Autor: Alexandre Teixeira
# Descrição: Este notebook realiza a preparação e modelagem de dados empresariais
#            para prever o valor total de pedidos por empresa no mês de abril de 2025,
#            utilizando regressão com HistGradientBoostingRegressor.
# Dataset: pedidos.csv, order_products.csv, prices.csv, users.csv, products.csv, companies.csv
# Data da última atualização: Julho/2025

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# Carregamento dos dados
orders = pd.read_csv('pedidos.csv')
order_products = pd.read_csv('order_products.csv')
prices = pd.read_csv('prices.csv')
users = pd.read_csv('users.csv')
products = pd.read_csv('products.csv')
companies = pd.read_csv('companies.csv')

# Conversão da coluna de data
orders['order_date'] = pd.to_datetime(orders['order_date'])

# Merge entre pedidos, produtos e preços
orders_products = pd.merge(order_products, orders, on='order_id')
orders_products_prices = pd.merge(orders_products, prices, on='product_id')

# Cálculo do valor total por item
orders_products_prices['total_value'] = orders_products_prices['quantity'] * orders_products_prices['price']

# Agregação por data e empresa
df = orders_products_prices.groupby(['order_date', 'company_id'])['total_value'].sum().reset_index()
df = df.sort_values(by='order_date')

# Criação de variáveis temporais
for i in range(1, 4):
    df[f'lag_{i}'] = df.groupby('company_id')['total_value'].shift(i)

df['diff_1'] = df['total_value'] - df['lag_1']
df['month'] = df['order_date'].dt.month
df['year'] = df['order_date'].dt.year

# Engenharia de atributos sazonais
monthly_avg = df.groupby(['company_id', 'month'])['total_value'].mean().reset_index()
monthly_avg.rename(columns={'total_value': 'monthly_avg'}, inplace=True)
df = pd.merge(df, monthly_avg, on=['company_id', 'month'], how='left')

df = df.dropna()

# Separação entre treino e teste (até março e abril de 2025)
df_treino = df[df['order_date'] < '2025-04-01']
df_teste = df[df['order_date'] >= '2025-04-01']

X_treino = df_treino.drop(columns=['order_date', 'total_value'])
y_treino = df_treino['total_value']

X_teste = df_teste.drop(columns=['order_date', 'total_value'])
y_teste = df_teste['total_value']

# Padronização
scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

# Treinamento do modelo
modelo = HistGradientBoostingRegressor()
modelo.fit(X_treino_scaled, y_treino)

# Avaliação da performance com RMSE
predicoes = modelo.predict(X_teste_scaled)
rmse = sqrt(mean_squared_error(y_teste, predicoes))
print("📈 RMSE do modelo (treinado até Março/2025):", rmse)

# Previsão do valor total por empresa (abril/2025)
df_teste['predito'] = predicoes
previsao_abril_agrupada = df_teste.groupby('company_id')['predito'].sum().reset_index()
print("🔮 Previsão de valor total por empresa - Abril/2025")
print(previsao_abril_agrupada